neste algoritmo dividimos o dataset afim de não bater o limite de requisição da API

In [ ]:
import pandas as pd
import requests
import json
import polyline
import folium
import numpy as np
from concorde.problem import Problem
from concorde.concorde import Concorde

# Carregar os dados do arquivo CSV
df = pd.read_csv("data.csv")
coordinates = df[["lon", "lat"]].values.tolist()
names = df["escolas_postos"].values.tolist()


# Define a função para solicitar as direções para um grupo de coordenadas
def get_directions_for_coordinates(coordinates):
    body = {"coordinates": coordinates}
    headers = {
        "Accept": "application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8",
        "Authorization": "",
        "Content-Type": "application/json; charset=utf-8",
    }
    response = requests.post(
        "https://api.openrouteservice.org/v2/directions/driving-hgv",
        json=body,
        headers=headers,
    )
    return response.json()


# Divida as coordenadas em lotes de tamanho desejado
batch_size = 5  # Número de coordenadas em cada solicitação
coordinates_batches = [
    coordinates[i : i + batch_size] for i in range(0, len(coordinates), batch_size)
]

# Inicialize um dicionário para armazenar as direções para cada lote de coordenadas
directions_data = {}

# Faça solicitações para cada lote de coordenadas e armazene os resultados
for i, batch in enumerate(coordinates_batches):
    directions_data[i] = get_directions_for_coordinates(batch)

# Inicialize o mapa
m = folium.Map(location=[-22.89533, -43.20957], zoom_start=13)

# Inicialize uma lista para as coordenadas da rota completa
route_coordinates = []

# Processar e visualizar os resultados
for i, batch in enumerate(coordinates_batches):
    directions = directions_data[i]

    # Decodificar a geometria da rota
    encoded_geometry = directions["routes"][0]["geometry"]
    decoded_geometry = polyline.decode(encoded_geometry)

    # Adicionar as coordenadas da rota ao mapa
    folium.PolyLine(
        locations=decoded_geometry,
        color="blue",
        weight=5,
        opacity=0.7,
    ).add_to(m)

    # Adicionar as coordenadas da rota ao conjunto completo
    route_coordinates.extend(decoded_geometry)

# Adicionar marcadores para as coordenadas originais
coordinates = np.array(coordinates)
for i, name in enumerate(names):
    folium.Marker(
        location=coordinates[i],
        popup=name,
        icon=folium.Icon(color="green", icon="info-sign"),
    ).add_to(m)

# Converter as coordenadas da rota completa para um objeto GeoJSON
route_geojson = {"type": "LineString", "coordinates": route_coordinates}

# Adicionar a rota completa ao mapa
route_layer = folium.GeoJson(
    route_geojson, name="Route", style_function=lambda x: {"color": "blue", "weight": 5}
).add_to(m)

# Adicionar um controle de camadas para ativar/desativar a rota
folium.LayerControl().add_to(m)

# Salvar o mapa em um arquivo HTML
m.save("route_map.html")
